In [20]:
import problog
import pandas as pd

from problog.program import PrologString
from problog.core import ProbLog
from problog import get_evaluatable
from problog.logic import Var, Term
from problog.program import SimpleProgram

TRIPLE = Term('triple')
RELATIONS = _read_relations('/home/karel/Documents/github/thesis/reasoner/test/test_data/relation_ids.del')

for key, value in RELATIONS.items():
    if value == 'subClassOf':
        SUBCLASS = Term(str(key))
    elif value == 'type':
        TYPE_ = Term(str(key))
    elif value == 'domain':
        DOMAIN_ = Term(str(key))
    elif value == 'range':
        RANGE_ = Term(str(key))
    elif value == 'subPropertyOf':
        SUBPROPERTY = Term(str(key))
    else:
        pass

def _read_relations(file_path):
    rel_dict = {}
    with open(file_path) as fin:
        for line in fin:    
            k,v=line.split(sep=None,maxsplit=1)
            rel_dict[k]=v.rstrip()
    return rel_dict

def reasoner_rules(triples):

    query = Term('query')

    C, C3, C2, C1, X, Y, Z, R, R1, R2, R3 = Var('C'), Var('C3'), Var('C2'), Var('C1'), Var('X'), Var('Y'), Var('Z'), Var('R'), \
                                            Var('R1'), Var('R2'), Var('R3')
    rdfs2 = TRIPLE(X, TYPE_, C) << ( TRIPLE(R, DOMAIN_, C) & TRIPLE(X, R, Y) )
    rdfs3 = TRIPLE(Y, TYPE_, C) << ( TRIPLE(R, RANGE_, C) & TRIPLE(X, R, Y) )
    rdfs5 = TRIPLE(R1, SUBPROPERTY, R3) << ( TRIPLE(R1, SUBPROPERTY, R2) & TRIPLE(R2, SUBPROPERTY, R2) )
    rdfs7 = TRIPLE(X, R2, Y) << ( TRIPLE(R1, SUBPROPERTY, R2) & TRIPLE(X, R1, Y) )
    rdfs9 = TRIPLE(X, TYPE_, C2) << ( TRIPLE(C1, SUBCLASS, C2) & TRIPLE(X, TYPE_, C1) )
    rdfs11 = TRIPLE(C1, SUBCLASS, C3) << ( TRIPLE(C1, SUBCLASS, C2) & TRIPLE(C2, SUBCLASS, C3) )

    p = SimpleProgram()

    for t in triples:
        p += t 

    p += rdfs2
    p += rdfs3
    p += rdfs5
    p += rdfs7
    p += rdfs9
    p += rdfs11

    p += query(TRIPLE(X, TYPE_, C))
    p += query(TRIPLE(Y, TYPE_, C))
    p += query(TRIPLE(R1, SUBPROPERTY, R3))
    p += query(TRIPLE(X, R2, Y))
    p += query(TRIPLE(X, TYPE_, C2))
    p += query(TRIPLE(C1, SUBCLASS, C3))

    result = get_evaluatable().create_from(p).evaluate()
    # for it in result.items() :
    #     print ('%s : %s' % (it))
    return result

In [21]:
def _read_triple(file_path):
    triples = []
    with open(file_path) as fin:
        for line in fin:
            h, r, t = line.strip().split('\t')
            triples.append(TRIPLE(Term(h), Term(r), Term(t),p=1.0))
    return triples


In [22]:
triples = _read_triple("/home/karel/Documents/github/thesis/reasoner/test/test_data/train.del")
print(len(RELATIONS))
print(len(triples))


94
21078


In [23]:
res = reasoner_rules(triples)
res

In [7]:
ids = {}
with open("/home/karel/Documents/github/thesis/reasoner/test/test_data/entity_ids.del") as f:
    for line in f:
       (k, v) = line.split()
       ids[int(k)] = v
print(ids)

relations = {}
with open("/home/karel/Documents/github/thesis/reasoner/test/test_data/relation_ids.del") as f:
    for line in f:
       (k, v) = line.split()
       ids[int(k)] = v
print(ids)

def translate_tripes_to_strings(triple):
    for key in triple:
        S, P, O = str(key).replace("triple(","").replace(")","").split(",")
        print(ids[int(S)], ids[int(P)], ids[int(O)])

{7: 'steakhouse', 8: 'restaurant', 9: 'foodEstablishment', 10: 'old_homestead_steakhouse', 11: 'chelsea_manhattan', 1: 'containedInPlace', 13: 'place', 6: 'affiliation', 15: 'memberOf', 16: 'valentyn_symonenko', 17: 'kyiv_national_university_of_trade_and_economics'}
{7: 'steakhouse', 8: 'restaurant', 9: 'foodEstablishment', 10: 'old_homestead_steakhouse', 11: 'chelsea_manhattan', 1: 'containedInPlace', 13: 'place', 6: 'affiliation', 15: 'memberOf', 16: 'valentyn_symonenko', 17: 'kyiv_national_university_of_trade_and_economics', 0: 'subclass', 2: 'type', 3: 'domain', 4: 'range', 5: 'subproperty'}


In [8]:
translate_tripes_to_strings(res)

valentyn_symonenko affiliation kyiv_national_university_of_trade_and_economics
affiliation subproperty memberOf
valentyn_symonenko memberOf kyiv_national_university_of_trade_and_economics
containedInPlace domain place
containedInPlace range place
old_homestead_steakhouse containedInPlace chelsea_manhattan
chelsea_manhattan type place
old_homestead_steakhouse type place
steakhouse subclass restaurant
restaurant subclass foodEstablishment
steakhouse subclass foodEstablishment
old_homestead_steakhouse type steakhouse
old_homestead_steakhouse type restaurant
old_homestead_steakhouse type foodEstablishment
